Imports

In [37]:
import warnings
warnings.filterwarnings('ignore')

Extract Atrophy Within Region of Interest

___

## Neuroimaging File Extraction

Here's a breakdown of the inputs:

### 0. Base Directorty (CSF)
- **base_directory**: Absolute path to the base directory containing CSF files.

### 1. Cerebrospinal Fluid (CSF)
- **glob_name_pattern**: File pattern to search for CSF files.

### 2. Grey Matter
- **glob_name_pattern**: File pattern to search for grey matter files.

### 3. White Matter
- **glob_name_pattern**: File pattern to search for white matter files.

---

**Instructions**: Please fill out the `base_directory` and `glob_name_pattern` fields for each segment. This will ensure that the extraction process can locate and identify the appropriate neuroimaging files for further analysis.

---

Input Directory
- This is the BIDS-style directory which was created by Notebook 02

In [38]:
# Shared Base Directory
base_directory = r'/Volumes/Expansion/datasets/adni/neuroimaging/all_patients_atrophy_seeds'

This is the Glob-style path to the subfolder containing niftis of interest
- For example, from the base_directory, */tissue_segment_z_scores will look for all subjects, all session folders within subjects, and then check the tissue_segment_z_scores folder. 


In [39]:
shared_glob_path = r'*/*/thresholded_tissue_segment_z_scores'

This is the list of tissue types to use from within each terminal folder. 
- tissue_to_import = ['cerebrospinal_fluid', 'white_matter', 'grey_matter']
- **Leave as unchanged if you are unsure**

In [40]:
tissue_to_import = ['*_csf-plus-gm.nii']

In [41]:
from calvin_utils.vbm_utils.loading import import_dataframes_by_tissue
imported_dataframes_by_tissue = import_dataframes_by_tissue(base_directory, shared_glob_path, tissue_to_import)

I will search:  /Volumes/Expansion/datasets/adni/neuroimaging/all_patients_atrophy_seeds/*/*/thresholded_tissue_segment_z_scores/**_csf-plus-gm.nii*
Imported data *_csf-plus-gm.nii data with 902629 voxels and 1305 patients
Example filename per subject: sub-037_S_4432_csf-plus-gm.nii

--------------------------------



**Extract Subject ID From File Names**
Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID.
2) Proceeding the subject ID. 

This Should Often Be Left Default

In [42]:
preceding_id = 'sub-'
proceeding_id = '.nii'

In [43]:
from calvin_utils.vbm_utils.preprocessing import rename_dataframe_subjects
thresholded_atrophy_df_dict = rename_dataframe_subjects(imported_dataframes_by_tissue, preceding_id, proceeding_id)

Dataframe:  *_csf-plus-gm.nii
        037_S_4432_csf-plus-gm  128_S_0258_csf-plus-gm  \
0                          0.0                     0.0   
1                          0.0                     0.0   
2                          0.0                     0.0   
3                          0.0                     0.0   
4                          0.0                     0.0   
...                        ...                     ...   
902624                     0.0                     0.0   
902625                     0.0                     0.0   
902626                     0.0                     0.0   
902627                     0.0                     0.0   
902628                     0.0                     0.0   

        062_S_0690_csf-plus-gm  135_S_4710_csf-plus-gm  \
0                          0.0                     0.0   
1                          0.0                     0.0   
2                          0.0                     0.0   
3                          0.0           

# Extract Region of Interest Atrophy

Import Region of Interest Masks

In [44]:
folder_to_import_from = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/VBM/rois'
shared_glob_pattern = '*.nii*'

In [45]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder 
region_of_interest_df = import_matrices_from_folder(connectivity_path=folder_to_import_from, file_pattern=shared_glob_pattern)
region_of_interest_df

I will search:  /Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/VBM/rois/*.nii*


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


Extract Damage Scores Per Region of Interest

In [46]:
from calvin_utils.vbm_utils.postprocessing import calculate_damage_scores
damage_df_dict = calculate_damage_scores(thresholded_atrophy_df_dict, region_of_interest_df)

Dataframe:  *_csf-plus-gm.nii
                       mni_Cerebellum.nii.gz mni_152_cortex_mask_2mm.nii  \
037_S_4432_csf-plus-gm         -38137.010026              -214938.853798   
128_S_0258_csf-plus-gm          -66628.39697              -300480.368619   
062_S_0690_csf-plus-gm         -46840.250393              -332045.279806   
135_S_4710_csf-plus-gm         -29142.136675              -225621.701975   
130_S_4997_csf-plus-gm         -30933.446638              -245986.365881   
...                                      ...                         ...   
053_S_4578_csf-plus-gm          -35690.99356              -222879.665168   
130_S_1200_csf-plus-gm         -58321.676623               -312999.49086   
011_S_0362_csf-plus-gm         -24985.327179              -192360.302385   
053_S_5208_csf-plus-gm          -41923.88451               -336908.89621   
116_S_4855_csf-plus-gm         -61911.343181              -386536.539527   

                       subcortex_mask_2mm.nii.gz mni_Temp

Extract Total Atrophy Volume

In [47]:
from calvin_utils.vbm_utils.postprocessing import calculate_total_atrophy_voxels
damage_df_dict = calculate_total_atrophy_voxels(thresholded_atrophy_df_dict, damage_df_dict)

Dataframe:  *_csf-plus-gm.nii


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,...,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz,Total Atrophy Voxels
037_S_4432_csf-plus-gm,-38137.010026,-214938.853798,-43963.187914,-60901.166534,-313266.562631,-152.609917,-34452.199912,-107356.828962,-49889.461371,-6478.72627,...,34067.0,8359.0,9017.0,51809.0,11441.0,6074.0,17018.0,9235.0,788.0,50040.0
128_S_0258_csf-plus-gm,-66628.39697,-300480.368619,-55181.412711,-84134.230511,-432369.788501,-1809.178075,-60695.298196,-118462.066258,-79179.767689,-7845.606727,...,42315.0,9539.0,11245.0,64082.0,15815.0,8666.0,17639.0,12027.0,926.0,61581.0
062_S_0690_csf-plus-gm,-46840.250393,-332045.279806,-52666.974133,-96350.599398,-452979.498675,-2396.054331,-60260.235886,-124925.619791,-96648.084485,-6683.926724,...,43242.0,9590.0,11564.0,64040.0,14556.0,8561.0,17605.0,13511.0,745.0,61325.0
135_S_4710_csf-plus-gm,-29142.136675,-225621.701975,-41829.381432,-50531.252864,-316365.055548,-479.294482,-46489.050651,-97472.897444,-72238.255875,-4359.584327,...,35979.0,7994.0,8240.0,52225.0,10390.0,7490.0,15923.0,11822.0,513.0,50454.0
130_S_4997_csf-plus-gm,-30933.446638,-245986.365881,-45468.987999,-65262.091536,-334010.072765,-182.569467,-38779.103097,-105357.603521,-78186.096702,-4778.105339,...,38970.0,8829.0,10065.0,56301.0,10613.0,7131.0,16795.0,13084.0,614.0,54884.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
053_S_4578_csf-plus-gm,-35690.99356,-222879.665168,-48194.860931,-58229.542325,-314025.311752,-199.888784,-45775.283822,-96917.669355,-60372.818541,-5187.128797,...,35273.0,8971.0,9107.0,52841.0,10412.0,7244.0,15658.0,10675.0,666.0,51032.0
130_S_1200_csf-plus-gm,-58321.676623,-312999.49086,-50535.903073,-83626.659515,-444939.538249,-1802.47998,-78758.676753,-105024.88508,-95269.192377,-6061.630919,...,41984.0,9357.0,10581.0,63155.0,14341.0,9719.0,16369.0,13388.0,723.0,60242.0
011_S_0362_csf-plus-gm,-24985.327179,-192360.302385,-45662.130534,-57634.854298,-277808.694518,-719.11392,-31801.101084,-86675.646872,-52790.327308,-3859.786877,...,35924.0,9083.0,10530.0,53095.0,10884.0,6103.0,16276.0,10375.0,656.0,51673.0
053_S_5208_csf-plus-gm,-41923.88451,-336908.89621,-55418.719089,-89338.223477,-464782.444518,-2788.890103,-51120.675617,-152136.921805,-92744.877449,-7768.406086,...,44661.0,9705.0,11397.0,65474.0,15168.0,7671.0,20098.0,13384.0,838.0,62828.0


------------- 



Organize the Subjects

In [48]:
from calvin_utils.vbm_utils.postprocessing import sort_dataframes_by_index
try:
    sorted_damage_df_dict = sort_dataframes_by_index(damage_df_dict)
except:
    sorted_damage_df_dict = damage_df_dict

Save the Results

In [49]:
damage_df_dict

{'*_csf-plus-gm.nii':                        mni_Cerebellum.nii.gz mni_152_cortex_mask_2mm.nii  \
 037_S_4432_csf-plus-gm         -38137.010026              -214938.853798   
 128_S_0258_csf-plus-gm          -66628.39697              -300480.368619   
 062_S_0690_csf-plus-gm         -46840.250393              -332045.279806   
 135_S_4710_csf-plus-gm         -29142.136675              -225621.701975   
 130_S_4997_csf-plus-gm         -30933.446638              -245986.365881   
 ...                                      ...                         ...   
 053_S_4578_csf-plus-gm          -35690.99356              -222879.665168   
 130_S_1200_csf-plus-gm         -58321.676623               -312999.49086   
 011_S_0362_csf-plus-gm         -24985.327179              -192360.302385   
 053_S_5208_csf-plus-gm          -41923.88451               -336908.89621   
 116_S_4855_csf-plus-gm         -61911.343181              -386536.539527   
 
                        subcortex_mask_2mm.nii.gz mni

In [50]:
analysis='0mm_zmap_atrophy_unthresholded_combo'

In [51]:
from calvin_utils.vbm_utils.postprocessing import save_csv_to_bids
save_csv_to_bids(sorted_damage_df_dict, bids_base_dir=base_directory, analysis=analysis, ses=None, dry_run=False)

100%|██████████| 1/1 [00:00<00:00, 10.85it/s]

Saved to:  /Volumes/Expansion/datasets/adni/neuroimaging/all_patients_atrophy_seeds/neuroimaging_analyses/ses-01/sub-all/0mm_zmap_atrophy_unthresholded_combo/*_csf-plus-gm.nii.csv


All done. Enjoy your analyses. 

--Calvin 